# Data Preparation Part 2

As mentioned in the **README** section, one of the most commonly used techniques for implementing machine learning models with sports data is to calculate averages based on past data and use them to predict match outcomes.

Various types of averages can be implemented. Here, we will use the **exponential moving average (EMA)**, as we believe it better captures the progress of a team throughout the season.

To achieve this, it is necessary to first convert the dataset into a **multiline format**, making the calculations for exponential moving averages computationally simpler and easier to write in Python code. Once the EMA calculations are completed, we will revert to the **original dataset format**.

From this code, we will generate the **EMA_data** dataset, which contains most of the features that will be used for applying machine learning models in the next section. As previously mentioned, these variables have been explained in detail in the **Dataset_explanation.txt** document.














In [3]:
from DataScraper import*
import pandas as pd
from datetime import datetime
import numpy as np
import os
import csv
%matplotlib inline

DATA_PATH = 'data'

## Creation of EMA_data dataset


In this initial phase, we combined all the seasonal data into a single comprehensive dataset called **`all_seasons_joined`** and exported it as a CSV file to highlight the total volume of data collected. 

Subsequently, we applied a series of functions to process this CSV file, transforming it step by step into a refined dataset containing **Exponential Moving Average (EMA)** data.







In [5]:
seasons_data = download_serie_a_data_by_season(start_season="1617", end_season="2425", output_folder="data")

df17_new = seasons_data['1617']
df18_new = seasons_data['1718']
df19_new = seasons_data['1819']
df20_new = seasons_data['1920']
df21_new = seasons_data['2021']
df22_new = seasons_data['2122']
df23_new = seasons_data['2223']
df24_new = seasons_data['2324']
df25_new = seasons_data['2425']


df25_new = df25_new[:80]     # to solve the problem as before. We make take the mathces until the last moment
                               # every team played same number of matches, otherwise our analysis would be biased
                               # towards team with more matches played. We stop at the 8th matchweek

In [6]:
df_new = pd.concat([df17_new, df18_new, df19_new, df20_new, df21_new, df22_new, df23_new, df24_new, df25_new],
                ignore_index=True, sort=False)


df_new['number'] = range(1, len(df_new) + 1)
df_new = df_new[['number'] + df_new.columns[:-1].tolist()]
df_new.head()

C:\Users\giuse\AppData\Local\Temp\ipykernel_7964\1884393545.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_new['number'] = range(1, len(df_new) + 1)


,number,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,1XBCH,1XBCD,1XBCA,BFECH,BFECD,BFECA,BFEC>2.5,BFEC<2.5,BFECAHH,BFECAHA
0,1,I1,20/08/16,Juventus,Fiorentina,2,1,H,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I1,20/08/16,Roma,Udinese,4,0,H,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,I1,21/08/16,Atalanta,Lazio,3,4,A,0.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,I1,21/08/16,Bologna,Crotone,1,0,H,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,I1,21/08/16,Chievo,Inter,2,0,H,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_new.to_csv(os.path.join(DATA_PATH, 'all_seasons_joined.csv'), index=False)

## Code Logic

### Function: `create_df(path)`
This function performs the following steps:
1. Automatically imports the DataFrame `all_seasons_joined.csv` from the specified `path`.
2. Converts the `Date` column to **datetime** format.
3. Removes columns with too many missing values and drops rows with missing values.
4. Renames the column `number` to `gameId`.
5. Sorts the data by `gameId` and resets the index.
6. Returns the cleaned and ordered Daame.

---

### Function: `create_multiline_df_stats(old_stats_df)`
This function transforms a **single-line-per-match** DataFrame into a **multi-line-per-team** DataFrame:
1. Defines specific columns for home and away team statistics and maps them to a unified column naming scheme.
2. Creates separate DataFrames for the home and away teams, renames their columns accordingly, and adds a flag (`homeGame`) to differentiate between home and away matches.
3. Concatenates the two DataFrames, sorts them by `gameId`, and resets the index.
4. Handles missing columns with an error-catch mechanism, reporting any is encountered.

---

### Function: `create_stats_features_ema(stats, span)`
This function calculates **Exponential Moving Average (EMA)** features for each team based on past match statistics:
1. Converts the input dataset into a **multiline format** (each row representing one team per match) using the `create_multiline_df_stats` function.
2. Creates a new DataFrame (`ema_features`) with basic columns such as `date`, `season`, `team`, and game details.
3. Identifies numeric feature columns to process and calculates the EMA for each feature, grouping the data by team.
4. Computes EMA values using the specified `span` and shifts the results by one row to exclude the current match from the calculation.
5. Returns the final DataFrame coining the EMA features.

---


In [9]:
def create_df(path):
    """
    Function to convert date to datetime and sort by gameId
    """
    df = (pd.read_csv(path, dtype={'season': str})
         .assign(Date=lambda df: pd.to_datetime(df.Date))
         .pipe(lambda df: df.dropna(thresh=len(df) - 2, axis=1))  # Drop cols with NAs
         .dropna(axis=0)  # Drop rows with NAs
         .rename(columns={'number': 'gameId'})
         .sort_values('gameId')
         .reset_index(drop=True)
         )
    return df

df = create_df(os.path.join(DATA_PATH, 'all_seasons_joined.csv'))
df.shape

C:\Users\giuse\AppData\Local\Temp\ipykernel_7964\1895271443.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  .assign(Date=lambda df: pd.to_datetime(df.Date))


(3117, 30)

In [10]:
def create_multiline_df_stats(old_stats_df):
    """
    Function to create a multiline DataFrame with individual rows for home and away teams.
    """
    # Columns for home and away team statistics
    home_stats_cols = ['Date', 'Season', 'HomeTeam', 'FTHG', 'FTAG', 'HTHG', 'HTAG', 'HS', 'AS', 'HST', 'AST', 
                       'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR']
    away_stats_cols = ['Date', 'Season', 'AwayTeam', 'FTAG', 'FTHG', 'HTAG', 'HTHG', 'AS', 'HS', 'AST', 'HST', 
                       'AF', 'HF', 'AC', 'HC', 'AY', 'HY', 'AR', 'HR']
    stats_cols_mapping = ['Date', 'Season', 'Team', 'goalsFor', 'goalsAgainst', 'halfTimeGoalsFor', 'halfTimeGoalsAgainst', 
                          'shotsFor', 'shotsAgainst', 'shotsOnTargetFor', 'shotsOnTargetAgainst', 'freesFor', 'freesAgainst', 
                          'cornersFor', 'cornersAgainst', 'yellowsFor', 'yellowsAgainst', 'redsFor', 'redsAgainst']

    # Column mapping
    home_mapping = {old_col: new_col for old_col, new_col in zip(home_stats_cols, stats_cols_mapping)}
    away_mapping = {old_col: new_col for old_col, new_col in zip(away_stats_cols, stats_cols_mapping)}

    # Creation of the multiline DataFrame
    try:
        home_stats = (old_stats_df[['gameId'] + home_stats_cols]
                      .rename(columns=home_mapping)
                      .assign(homeGame=1))
        print("Home stats processed successfully.")

        away_stats = (old_stats_df[['gameId'] + away_stats_cols]
                      .rename(columns=away_mapping)
                      .assign(homeGame=0))
        print("Away stats processed successfully.")

        multi_line_stats = (pd.concat([home_stats, away_stats], axis=0, sort=True)
                            .sort_values(by='gameId')
                            .reset_index(drop=True))
        print(f"Multiline DataFrame created with {multi_line_stats.shape[0]} rows and {multi_line_stats.shape[1]} columns.")
        return multi_line_stats
    except KeyError as e:
        print(f"Missing column in the dataset: {e}")
        return None


In [11]:
def create_stats_features_ema(stats, span):
    """
    Function to create EMA features for each team.
    """
    multi_line_stats = create_multiline_df_stats(stats)
    assert multi_line_stats is not None, "The multiline DataFrame is None. Check the input dataset."

    # Create an empty DataFrame for EMA
    ema_features = multi_line_stats[['Date', 'Season', 'gameId', 'Team', 'homeGame']].copy()

    # Get the column names to process
    feature_names = multi_line_stats.drop(columns=['Date', 'Season', 'gameId', 'Team', 'homeGame']).columns

    # Calculate EMA for each feature
    for feature_name in feature_names:
        ema_features[feature_name] = (multi_line_stats.groupby('Team')[feature_name]
                                      .transform(lambda row: row.ewm(span=span, min_periods=2).mean().shift(1)))   # Minimum periods required for EWM computation
        print(f"Processed EMA for feature: {feature_name}")
    print("EMA features created successfully.")
    return ema_features



We have decided to consider a span of **10** to ensure the model focuses on the most recent performance trends. Football teams often experience significant changes in form due to factors such as injuries, tactical adjustments, or player availability, making recent data more relevant than older matches. 

This approach captures current dynamics, such as **momentum** and **confidence**, while reducing the noise introduced by outdated information. A shorter span provides a better reflection of the team’s present state and adapts more effectively to **tactical adjustments** or **roster changes**.

By prioritizing recent data, we aim to make the analysis more accurate and actionable for predicting upcoming matches.







In [13]:
# Add weighted average to each row with a span of 10.
df_ema = create_stats_features_ema(df, span=10)
df_ema.head()

Home stats processed successfully.
Away stats processed successfully.
Multiline DataFrame created with 6234 rows and 21 columns.
Processed EMA for feature: cornersAgainst
Processed EMA for feature: cornersFor
Processed EMA for feature: freesAgainst
Processed EMA for feature: freesFor
Processed EMA for feature: goalsAgainst
Processed EMA for feature: goalsFor
Processed EMA for feature: halfTimeGoalsAgainst
Processed EMA for feature: halfTimeGoalsFor
Processed EMA for feature: redsAgainst
Processed EMA for feature: redsFor
Processed EMA for feature: shotsAgainst
Processed EMA for feature: shotsFor
Processed EMA for feature: shotsOnTargetAgainst
Processed EMA for feature: shotsOnTargetFor
Processed EMA for feature: yellowsAgainst
Processed EMA for feature: yellowsFor
EMA features created successfully.


,Date,Season,gameId,Team,homeGame,cornersAgainst,cornersFor,freesAgainst,freesFor,goalsAgainst,...,halfTimeGoalsAgainst,halfTimeGoalsFor,redsAgainst,redsFor,shotsAgainst,shotsFor,shotsOnTargetAgainst,shotsOnTargetFor,yellowsAgainst,yellowsFor
0,2016-08-20,1617,1,Juventus,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-08-20,1617,1,Fiorentina,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-08-20,1617,2,Roma,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-08-20,1617,2,Udinese,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-08-21,1617,3,Atalanta,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We have chosen 50 in order to better capture the longer term teams growth or decline path

In [15]:
df_ema.isnull().any(axis=1).sum()

66


Now we observe that there are **66 rows with NaN values**. This is entirely expected due to how we set up the EMA computation. Specifically, the EMA for each feature is calculated only after a team appears in the dataset at least twice (see function: `create_stats_features_ema`). As a result, the EMA values for any team will begin from the **third instance** of that team in the multiline DataFrame. 

Consequently, the presence of 66 missing data points is reasonable, particularly for **newly promoted teams** at the start of each season.

With the EMA stats added, we can now restructure the dataset to have **one row per match**. Finally, we save it as a **CSV file** for further analysis.







In [17]:
def restructure_stats_features(stats_features):
    non_features = ['homeGame', 'Team', 'gameId']

    stats_features_restructured = (stats_features.query('homeGame == 1')
                                    .rename(columns={col: 'f_' + col + 'Home' for col in stats_features.columns if col not in non_features})
                                    .rename(columns={'Team': 'HomeTeam'})
                                    .pipe(pd.merge, (stats_features.query('homeGame == 0')
                                                        .rename(columns={'Team': 'AwayTeam'})
                                                        .rename(columns={col: 'f_' + col + 'Away' for col in stats_features.columns 
                                                                         if col not in non_features})), on=['gameId'])
                                    .dropna())
    return stats_features_restructured

df_ema_final = restructure_stats_features(df_ema)
df_ema_final.tail()

,f_DateHome,f_SeasonHome,gameId,HomeTeam,homeGame_x,f_cornersAgainstHome,f_cornersForHome,f_freesAgainstHome,f_freesForHome,f_goalsAgainstHome,...,f_halfTimeGoalsAgainstAway,f_halfTimeGoalsForAway,f_redsAgainstAway,f_redsForAway,f_shotsAgainstAway,f_shotsForAway,f_shotsOnTargetAgainstAway,f_shotsOnTargetForAway,f_yellowsAgainstAway,f_yellowsForAway
3112,2024-10-20,2425,3116,Lecce,1,5.949063,4.327636,11.667950,12.196196,1.498062,...,0.835163,0.771618,0.187666,0.054997,11.861504,13.153325,4.682292,4.268263,2.261853,2.022254
3113,2024-10-20,2425,3117,Venezia,1,7.734226,3.508483,11.996454,11.001345,1.695400,...,0.606449,1.095937,0.149224,0.000199,10.611429,16.771582,3.787876,5.834891,1.663594,1.659606
3114,2024-10-20,2425,3118,Cagliari,1,5.498134,5.344111,12.558534,13.736455,1.776324,...,0.807537,0.638397,0.167516,0.220112,17.080906,10.258752,5.552664,4.050801,2.261122,1.986619
3115,2024-10-20,2425,3119,Roma,1,3.522885,4.360449,12.503509,11.162522,0.881165,...,0.673632,1.281323,0.236926,0.016379,10.279474,17.192867,4.329257,5.806002,1.383074,1.162247
3116,2024-10-21,2425,3120,Verona,1,5.729701,5.437487,9.799404,13.503394,1.734489,...,0.781684,0.355927,0.003430,0.053792,13.604644,6.440977,4.027350,2.425707,1.670801,2.401249


In [18]:
df_ema_final.columns

Index(['f_DateHome', 'f_SeasonHome', 'gameId', 'HomeTeam', 'homeGame_x',
       'f_cornersAgainstHome', 'f_cornersForHome', 'f_freesAgainstHome',
       'f_freesForHome', 'f_goalsAgainstHome', 'f_goalsForHome',
       'f_halfTimeGoalsAgainstHome', 'f_halfTimeGoalsForHome',
       'f_redsAgainstHome', 'f_redsForHome', 'f_shotsAgainstHome',
       'f_shotsForHome', 'f_shotsOnTargetAgainstHome',
       'f_shotsOnTargetForHome', 'f_yellowsAgainstHome', 'f_yellowsForHome',
       'f_DateAway', 'f_SeasonAway', 'AwayTeam', 'homeGame_y',
       'f_cornersAgainstAway', 'f_cornersForAway', 'f_freesAgainstAway',
       'f_freesForAway', 'f_goalsAgainstAway', 'f_goalsForAway',
       'f_halfTimeGoalsAgainstAway', 'f_halfTimeGoalsForAway',
       'f_redsAgainstAway', 'f_redsForAway', 'f_shotsAgainstAway',
       'f_shotsForAway', 'f_shotsOnTargetAgainstAway',
       'f_shotsOnTargetForAway', 'f_yellowsAgainstAway', 'f_yellowsForAway'],
      dtype='object')

In [19]:
df_ema_final[df_ema_final.isna().any(axis=1)]

,f_DateHome,f_SeasonHome,gameId,HomeTeam,homeGame_x,f_cornersAgainstHome,f_cornersForHome,f_freesAgainstHome,f_freesForHome,f_goalsAgainstHome,...,f_halfTimeGoalsAgainstAway,f_halfTimeGoalsForAway,f_redsAgainstAway,f_redsForAway,f_shotsAgainstAway,f_shotsForAway,f_shotsOnTargetAgainstAway,f_shotsOnTargetForAway,f_yellowsAgainstAway,f_yellowsForAway


In [20]:
df_ema_final.shape

(3070, 41)

In [21]:
df_ema_final.to_csv(os.path.join(DATA_PATH, 'EMA_data.csv'))

The newly created dataset is then saved in the "Data" folder.

--> Proceed to the "Data_analysis" section.